rbp conditional latent diffusion

### mount gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
os.chdir('/content/drive/MyDrive/rna-binding')

In [ ]:
!pip install transformers

### data prep

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('final_attract_db_with_emb.csv')

In [ ]:
import pickle
# Load the dictionary back from the pickle file.
with open("rbp_seqs_dict.pkl", "rb") as f:
    rbp_seqs_dict = pickle.load(f)

In [ ]:
import numpy as np
rna_motif_emb = np.load('rna_motif_emb.npy', allow_pickle=True)

In [ ]:
len(rna_motif_emb[1])

7

In [ ]:
data = data.drop(columns=['rna_motif_emb', 'rbp_esm_emb'])

In [ ]:
data['rna_motif_emb'] = rna_motif_emb

In [ ]:
data['rbp_esm_emb'] = data['RBP_sequence'].map(rbp_seqs_dict)

In [ ]:
# Convert list of tensors to numpy array
def tensors_to_numpy(tensor_list):
    return np.stack([t.numpy() for t in tensor_list])

# Apply the conversion to the 'rbp_esm_emb' column
data['rbp_esm_emb'] = data['rbp_esm_emb'].apply(tensors_to_numpy)

In [ ]:
average_length = data['RBP_sequence'].apply(len).mean()
print(f"Average length of RBP_sequence: {average_length}")


Average length of RBP_sequence: 415.5270602706027


In [ ]:
data = data[data['RBP_sequence'].apply(len) <= 750]
filtered_data = data.copy()

In [ ]:
max_rbp_sequence_length = data['RBP_sequence'].apply(len).max()
max_motif_length = data['Motif'].apply(len).max()

print(f"Maximum RBP sequence length: {max_rbp_sequence_length}")
print(f"Maximum Motif length: {max_motif_length}")

# Use these lengths for padding
max_encoder_seq_length = max_motif_length
max_decoder_seq_length = max_rbp_sequence_length

Maximum RBP sequence length: 741
Maximum Motif length: 12


In [ ]:
# import pandas as pd
# from scipy import stats

# # Assuming meta4k is a DataFrame and 'sequence' is the column with sequence strings
# # Calculate sequence lengths
# data['sequence_length'] = data['RBP_sequence'].apply(len)

# # Compute Z-scores of the lengths
# data['z_scores'] = stats.zscore(data['sequence_length'])

# # Filter the outliers; this will keep only the rows that are not outliers
# filtered_data = data[(data['z_scores'] > -3) & (data['z_scores'] < 3)]

# # Determine the outliers
# outliers = data[(data['z_scores'] <= -3) | (data['z_scores'] >= 3)]

# # Print the outliers
# print(outliers)


In [ ]:
max_rbp_sequence_length = filtered_data['RBP_sequence'].apply(len).max()
min_rbp_sequence_length = filtered_data['RBP_sequence'].apply(len).min()
max_motif_length = filtered_data['Motif'].apply(len).max()
min_motif_length = filtered_data['Motif'].apply(len).min()

print(f"Maximum RBP sequence length: {max_rbp_sequence_length}")
print(f"Maximum Motif length: {max_motif_length}")
print(f"Minimum RBP sequence length: {min_rbp_sequence_length}")
print(f"Minimum Motif length: {min_motif_length}")

# Use these lengths for padding
max_encoder_seq_length = max_motif_length
max_decoder_seq_length = max_rbp_sequence_length

Maximum RBP sequence length: 741
Maximum Motif length: 12
Minimum RBP sequence length: 136
Minimum Motif length: 4


In [ ]:
import numpy as np
import ast

### dataloader -- treat protein emebdding as the rna motif & treat molecular embedding as protein embedding

In [ ]:
data.columns

Index(['Gene_name', 'Gene_id', 'Motif', 'RBP_sequence', 'rna_motif_emb',
       'rbp_esm_emb'],
      dtype='object')

In [ ]:
print(data['rna_motif_emb'][1].shape)
print(data['rbp_esm_emb'][1].shape)

(7, 120)
(594, 1280)


In [ ]:
data = filtered_data.copy()

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Checking the GPU availability
if not torch.cuda.is_available():
    raise SystemError('GPU device not found')
print(f"Using device: {device}")


Using device: cuda


In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
!pip install fair-esm
##setting up ESM
import torch
import esm
esm_model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
esm_model.eval()  # disables dropout for deterministic results
esm_model.cuda() #push model to gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 2.3 MB/s eta 0:00:00


Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt


ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [ ]:
protein_input_list = [torch.tensor(seq, dtype=torch.float32) for seq in data['rna_motif_emb'].tolist()]
molecule_input_list = [torch.tensor(seq, dtype=torch.float32) for seq in data['rbp_esm_emb'].tolist()]
molecule_target_list = [torch.tensor(np.roll(seq, shift=-1, axis=0), dtype=torch.float32) for seq in data['rbp_esm_emb'].tolist()]

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence

class Seq2SeqDataset(Dataset):
    def __init__(self, encoder_data, decoder_data, decoder_target, max_protein_seq_length, max_molecule_seq_length):
        self.encoder_data = encoder_data
        self.decoder_data = decoder_data
        self.decoder_target = decoder_target
        self.max_protein_seq_length = max_protein_seq_length
        self.max_molecule_seq_length = max_molecule_seq_length

    def __len__(self):
        return len(self.encoder_data)

    def __getitem__(self, idx):
        encoder_seq = torch.nn.functional.pad(self.encoder_data[idx], (0, 0, 0, self.max_protein_seq_length - self.encoder_data[idx].size(0)))
        decoder_seq = torch.nn.functional.pad(self.decoder_data[idx], (0, 0, 0, self.max_molecule_seq_length - self.decoder_data[idx].size(0)))
        decoder_target = torch.nn.functional.pad(self.decoder_target[idx], (0, 0, 0, self.max_molecule_seq_length - self.decoder_target[idx].size(0)))
        return encoder_seq, decoder_seq, decoder_target

# Find the maximum sequence length for proteins and molecules
max_protein_seq_length = max(seq.size(0) for seq in protein_input_list)
max_molecule_seq_length = 750 #max(seq.size(0) for seq in molecule_input_list)

# Create the dataset
dataset = Seq2SeqDataset(protein_input_list, molecule_input_list, molecule_target_list, max_protein_seq_length, max_molecule_seq_length)

# Splitting the dataset into training and test/validation sets
train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # 20% for testing
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoaders for the training and testing sets
batch_size = 4  # You can adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
train_loader.dataset[20][0].shape

torch.Size([12, 120])

In [ ]:
train_dataset[0][1].shape

torch.Size([750, 1280])

### diffusion design module

transition module

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

variance scheduler

In [ ]:
class VarianceSchedule(nn.Module):

    def __init__(self, num_steps=100, s=0.01):
        super().__init__()
        T = num_steps
        t = torch.arange(0, num_steps+1, dtype=torch.float)
        f_t = torch.cos( (np.pi / 2) * ((t/T) + s) / (1 + s) ) ** 2
        alpha_bars = f_t / f_t[0]

        betas = 1 - (alpha_bars[1:] / alpha_bars[:-1])
        betas = torch.cat([torch.zeros([1]), betas], dim=0)
        betas = betas.clamp_max(0.999)

        sigmas = torch.zeros_like(betas)
        for i in range(1, betas.size(0)):
            sigmas[i] = ((1 - alpha_bars[i-1]) / (1 - alpha_bars[i])) * betas[i]
        sigmas = torch.sqrt(sigmas)

        self.register_buffer('betas', betas)
        self.register_buffer('alpha_bars', alpha_bars)
        self.register_buffer('alphas', 1 - betas)
        self.register_buffer('sigmas', sigmas)
        # calculate X0 = sqrt_recip_alphas_cumprod * Xt - sqrt_recipm1_alphas_cumprod * noise
        self.register_buffer('sqrt_recip_alphas_cumprod', torch.sqrt(1. / alpha_bars))
        self.register_buffer('sqrt_recipm1_alphas_cumprod', torch.sqrt(1. / alpha_bars - 1))

        alphas = 1 - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value = 1.)
        self.posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)
        self.posterior_log_variance_clipped = torch.log(self.posterior_variance.clamp(min =1e-20))
        self.posterior_mean_coef1 = betas * torch.sqrt(alphas_cumprod_prev) / (1. - alphas_cumprod)
        self.posterior_mean_coef2 = (1. - alphas_cumprod_prev) * torch.sqrt(alphas) / (1. - alphas_cumprod)

    def to(self,device):
        for k, v in self.__dict__.items():
            if isinstance(v, torch.Tensor):
                self.__dict__[k] = v.to(device)



rbp continuous transition -- adapted from small molecule version

In [ ]:
class ProteinEmbeddingTransformation(nn.Module):
    def __init__(self, protein_embedding_dim, output_embedding_dim, target_seq_length):
        super().__init__()
        self.target_seq_length = target_seq_length
        self.rnn = nn.GRU(protein_embedding_dim, output_embedding_dim, batch_first=True)
        self.fc_final = nn.Linear(output_embedding_dim, output_embedding_dim)

    def forward(self, protein_embedding):
        # GRU Layer expects input of shape (batch, seq_len, input_size)
        # Assuming protein_embedding is already in this shape
        output, _ = self.rnn(protein_embedding)

        # Taking the output corresponding to the last timestep
        last_output = output[:, -1, :]

        # Repeating the last output to match the target sequence length
        last_output_repeated = last_output.unsqueeze(1).repeat(1, self.target_seq_length, 1)

        # Passing through a final Linear layer
        transformed_embedding = self.fc_final(last_output_repeated)

        return transformed_embedding


class SmallMoleculeContinuousTransition(nn.Module):
    def __init__(self, num_steps, embedding_dim, protein_embedding_dim, noise_mean, noise_std, var_sched_opt={}):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.protein_embedding_dim = protein_embedding_dim
        self.var_sched = VarianceSchedule(num_steps, **var_sched_opt)
        self.noise_mean = noise_mean
        self.noise_std = noise_std

        # Layers for adding noise
        self.add_noise_layer = nn.Linear(embedding_dim, embedding_dim)

        # Denoising layers, now also conditioned on protein embeddings
        self.denoise_layer = nn.Linear(embedding_dim + protein_embedding_dim, embedding_dim)
        self.denoise_mlp = nn.Sequential(
            nn.Linear(embedding_dim + protein_embedding_dim, embedding_dim),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(embedding_dim, embedding_dim),
        )

    def add_noise(self, molecule_embedding, t):
        # Add noise based on the timestep 't'
        alpha_bar = self.var_sched.alpha_bars[t].unsqueeze(-1).unsqueeze(-1)
        noise = torch.randn_like(molecule_embedding) * self.noise_std + self.noise_mean
        noised_embedding = alpha_bar * molecule_embedding + (1 - alpha_bar) * self.add_noise_layer(noise)
        return noised_embedding

    def denoise(self, noised_embedding, protein_embedding, t):
        # Instantiate the transformation module
        transform_module = ProteinEmbeddingTransformation(
            protein_embedding_dim=self.protein_embedding_dim,
            output_embedding_dim=120,  # Adjust as needed
            target_seq_length=noised_embedding.size(1)  # This is 750 in your case
        ).to(protein_embedding.device)

        # Transform protein_embedding to match the sequence length
        transformed_protein_embedding = transform_module(protein_embedding)

        # Now concatenate with noised_embedding
        combined_embedding = torch.cat((noised_embedding, transformed_protein_embedding), dim=2)
        alpha_bar = self.var_sched.alpha_bars[t].unsqueeze(-1).unsqueeze(-1)
        processed_embedding = self.denoise_layer(combined_embedding)
        mlp_output = self.denoise_mlp(combined_embedding)
        predicted_embedding = alpha_bar * (mlp_output + processed_embedding) + (1 - alpha_bar) * noised_embedding
        return predicted_embedding


full_dpm & epsilon_net

In [ ]:
# class EpsilonNet(nn.Module):
#     def __init__(self, molecule_embedding_dim, protein_embedding_dim):
#         super().__init__()
#         self.fc1 = nn.Linear(protein_embedding_dim, 512)
#         self.fc2 = nn.Linear(512, 512)
#         self.fc3 = nn.Linear(512, molecule_embedding_dim)
#         self.relu = nn.ReLU()

#     def forward(self, protein_embedding):
#         x = self.relu(self.fc1(protein_embedding))
#         print('x after relu1 epsnet')
#         print(x.shape)
#         x = self.relu(self.fc2(x))
#         print('x after relu2 epsnet')
#         print(x.shape)
#         epsilon = self.fc3(x)
#         print('after fc3 epsnet -- EPSILON -- SHOUDL HAVE 1280 DIM')
#         print(epsilon.shape)
#         return epsilon

In [ ]:
# class EpsilonNet(nn.Module):
#     def __init__(self, molecule_embedding_dim, protein_embedding_dim, molecule_seqlength):
#         super().__init__()
#         self.molecule_seqlength = molecule_seqlength
#         self.fc1 = nn.GRU(protein_embedding_dim, 512, batch_first=True)
#         self.fc2 = nn.Linear(512, 512)
#         self.rnn = nn.GRU(512, molecule_embedding_dim, batch_first=True)
#         self.fc_final = nn.Linear(molecule_embedding_dim, molecule_embedding_dim)
#         self.relu = nn.ReLU()

#     def forward(self, protein_embedding):
#         # GRU Layer expects input of shape (batch, seq_len, input_size)
#         # Assuming protein_embedding is already in this shape
#         output, _ = self.fc1(protein_embedding)

#         # Process the output of GRU layer
#         # Taking the output corresponding to the last timestep
#         last_output = output[:, -1, :]

#         # Passing through Linear layer and ReLU
#         x = self.fc2(last_output)
#         x = self.relu(x)

#         # Preparing input for the second GRU layer
#         # Repeating x along the time dimension to match molecule_seqlength
#         x_repeated = x.unsqueeze(1).repeat(1, self.molecule_seqlength, 1)

#         # Second GRU layer
#         output, _ = self.rnn(x_repeated)

#         # Final Linear layer
#         epsilon = self.fc_final(output)

#         return epsilon


In [ ]:
import torch
import torch.nn as nn

class EpsilonNet(nn.Module):
    def __init__(self, molecule_embedding_dim, protein_embedding_dim, molecule_seqlength):
        super().__init__()
        self.molecule_seqlength = molecule_seqlength
        self.fc1 = nn.Linear(protein_embedding_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, molecule_embedding_dim)
        self.positional_encoding = nn.Parameter(torch.randn(1, molecule_seqlength, molecule_embedding_dim))
        self.relu = nn.ReLU()

    def forward(self, protein_embedding):
        # Flatten protein_embedding if it's 3D (batch, seq_len, input_size) to 2D (batch, input_size)
        if protein_embedding.dim() == 3:
            protein_embedding = protein_embedding.view(protein_embedding.size(0), -1)

        # Pass through fully connected layers
        x = self.fc1(protein_embedding)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)

        # Expand the output to match the molecule sequence length
        x = self.fc3(x)
        x = x.unsqueeze(1).expand(-1, self.molecule_seqlength, -1)

        # Add positional encodings to give each position in the sequence a unique signature
        x = x + self.positional_encoding

        return x


In [ ]:
class EpsilonNet(nn.Module):
    def __init__(self, molecule_embedding_dim, protein_embedding_dim, molecule_seqlength):
        super().__init__()
        self.molecule_seqlength = molecule_seqlength
        self.gru = nn.GRU(protein_embedding_dim, 512, batch_first=True)
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, molecule_embedding_dim)
        self.positional_encoding = nn.Parameter(torch.randn(1, molecule_seqlength, molecule_embedding_dim))
        self.relu = nn.ReLU()

    def forward(self, protein_embedding):
        # Process protein_embedding through GRU
        output, _ = self.gru(protein_embedding)

        # Use the output of the last timestep
        last_output = output[:, -1, :]

        # Pass through fully connected layers
        x = self.relu(self.fc1(last_output))
        x = self.fc2(x)

        # Expand the output to match the molecule sequence length
        x = x.unsqueeze(1).expand(-1, self.molecule_seqlength, -1)

        # Add positional encodings to give each position in the sequence a unique signature
        x = x + self.positional_encoding

        return x


In [ ]:
class FullDPM(nn.Module):
    def __init__(self, molecule_embedding_dim, protein_embedding_dim, num_steps, noise_mean, noise_std, molecule_seqlength):
        super().__init__()
        self.molecule_embedding_dim = molecule_embedding_dim
        self.protein_embedding_dim = protein_embedding_dim
        self.num_steps = num_steps
        self.molecule_seqlength = molecule_seqlength  # Add molecule_seqlength as an attribute
        self.epsilon_net = EpsilonNet(molecule_embedding_dim, protein_embedding_dim, molecule_seqlength)
        self.transition_model = SmallMoleculeContinuousTransition(num_steps, molecule_embedding_dim, protein_embedding_dim, noise_mean, noise_std)


    def forward(self, protein_embedding, t):
        # Start with noise
        molecule_seqlength = 750
        molecule_embedding = torch.randn(protein_embedding.size(0), molecule_seqlength, self.molecule_embedding_dim, device=protein_embedding.device)
        print('fulldpmforward-moleuclceembdding')
        print(molecule_embedding.shape)
        noised_embedding = self.transition_model.add_noise(molecule_embedding, t)
        print("fulldpm forward noised_embedding_done...")
        print('shape of protein_embedding before epsilon below...')
        print(protein_embedding.shape)
        epsilon = self.epsilon_net(protein_embedding)
        print('fulldpm-forward-noiseembedding')
        print(noised_embedding.shape)
        print('fulldpm-forward-epsilon')
        print(epsilon.shape)
        print('fulldpm-forward-proteinemebdding')
        print(protein_embedding.shape)
        print('fulldpm-forward-t')
        print(t)
        denoised_embedding = self.transition_model.denoise(noised_embedding + epsilon, protein_embedding, t)
        return denoised_embedding

    @torch.no_grad()
    def sample(self, protein_embedding):
        molecule_embedding = torch.randn(protein_embedding.size(0), self.molecule_embedding_dim, device=protein_embedding.device)
        for t in reversed(range(self.num_steps)):
            molecule_embedding = self.forward(protein_embedding, t)
        return molecule_embedding


design module

In [ ]:
class DiffusionProteinToMolecule(nn.Module):
    def __init__(self, cfg, num_protein_features, num_molecule_features):
        super().__init__()
        self.cfg = cfg

        self.diffusion = FullDPM(
            num_molecule_features,
            num_protein_features,
            cfg['num_steps'],
            cfg['noise_mean'],
            cfg['noise_std'],
            cfg['molecule_seqlength']  # Add this line
        )

    def forward(self, protein_emb, t=None):
        if t is None:
            t = torch.randint(0, self.cfg['num_steps'], (1,), dtype=torch.long)
        return self.diffusion(protein_emb, t)

    @torch.no_grad()
    def sample(self, protein_emb):
        return self.diffusion.sample(protein_emb)


In [ ]:
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Configuration for the diffusion model
cfg = {
    'num_steps': 100,  # Number of steps in the diffusion process
    'noise_mean': 0.0,
    'noise_std': 1.0,
    'molecule_seqlength': 750  # Adjust to the desired molecule sequence length
}


num_protein_features = 120  # dimension of motif embeddings
num_molecule_features = 1280  # output dimension of rbp embeddings

# Initialize the model
model = DiffusionProteinToMolecule(cfg, num_protein_features, num_molecule_features)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))


DiffusionProteinToMolecule(
  (diffusion): FullDPM(
    (epsilon_net): EpsilonNet(
      (fc1): Linear(in_features=120, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=512, bias=True)
      (fc3): Linear(in_features=512, out_features=1280, bias=True)
      (relu): ReLU()
    )
    (transition_model): SmallMoleculeContinuousTransition(
      (var_sched): VarianceSchedule()
      (add_noise_layer): Linear(in_features=1280, out_features=1280, bias=True)
      (denoise_layer): Linear(in_features=1400, out_features=1280, bias=True)
      (denoise_mlp): Sequential(
        (0): Linear(in_features=1400, out_features=1280, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.2, inplace=False)
        (3): Linear(in_features=1280, out_features=1280, bias=True)
      )
    )
  )
)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda:0


In [ ]:
train_loader.dataset[0][1].shape

torch.Size([750, 1280])

In [ ]:
import os
import sys


def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch_idx, (protein_emb, true_molecule_emb, *_) in enumerate(train_loader):
        protein_emb = protein_emb.to(device)
        print('proteinembshape')
        print(protein_emb.shape)
        print('moleculeembshape')
        print(true_molecule_emb.shape)
        true_molecule_emb = true_molecule_emb.to(device)
        optimizer.zero_grad()
        generated_molecule_emb = model(protein_emb)
        print('generatedmoleucleshape')
        print(generated_molecule_emb.shape)
        loss = criterion(generated_molecule_emb, true_molecule_emb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for protein_emb, true_molecule_emb, _ in val_loader:  # Unpack all three elements
            protein_emb = protein_emb.to(device)
            true_molecule_emb = true_molecule_emb.to(device)
            generated_molecule_emb = model(protein_emb)
            loss = criterion(generated_molecule_emb, true_molecule_emb)
            total_loss += loss.item()
    return total_loss / len(val_loader)


# Training setup
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # Adjust learning rate
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # use gradient clipping -- combats exploding gradients
# criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.MSELoss()
criterion = torch.nn.L1Loss()


# Training execution
import matplotlib.pyplot as plt

train_losses = []
val_losses = []

num_epochs = 20

for epoch in range(1, num_epochs + 1):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss = validate(model, test_loader, criterion, device)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")



proteinembshape
torch.Size([4, 12, 120])
moleculeembshape
torch.Size([4, 750, 1280])
fulldpmforward-moleuclceembdding
torch.Size([4, 750, 1280])
fulldpm forward noised_embedding_done...
shape of protein_embedding before epsilon below...
torch.Size([4, 12, 120])


RuntimeError: ignored

In [ ]:
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()
plt.show()

evaluating output embeddings

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch

model.eval()
generated_embeddings = []
ground_truth_embeddings = []

with torch.no_grad():
    for protein_emb, molecule_emb,_ in test_loader:
        protein_emb = protein_emb.to(device)
        molecule_emb = molecule_emb.to(device)
        output = model(protein_emb)
        generated_embeddings.extend(output.cpu().numpy())
        ground_truth_embeddings.extend(molecule_emb.cpu().numpy())

# Convert to numpy arrays
generated_embeddings = np.array(generated_embeddings)
ground_truth_embeddings = np.array(ground_truth_embeddings)
len(ground_truth_embeddings)

In [ ]:
# Run PCA
pca = PCA(n_components=2) # Change to n_components=3 for 3D
combined_embeddings = np.concatenate((generated_embeddings, ground_truth_embeddings))
pca_result = pca.fit_transform(combined_embeddings)

# Split the PCA result back into generated and ground truth
split_point = generated_embeddings.shape[0]
generated_pca = pca_result[:split_point, :]
ground_truth_pca = pca_result[split_point:, :]

# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(generated_pca[:, 0], generated_pca[:, 1], label='Generated Embeddings', alpha=0.5)
plt.scatter(ground_truth_pca[:, 0], ground_truth_pca[:, 1], label='Ground Truth Embeddings', alpha=0.5)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('PCA of Generated vs Ground Truth Embeddings')
plt.legend()
plt.show()

In [ ]:
from sklearn.manifold import TSNE

# Running t-SNE on the combined embeddings
tsne = TSNE(n_components=2, random_state=42)  # set random_state for reproducibility
tsne_result = tsne.fit_transform(combined_embeddings)

# Split the t-SNE result back into generated and ground truth embeddings
generated_tsne = tsne_result[:split_point, :]
ground_truth_tsne = tsne_result[split_point:, :]

# Plotting the t-SNE results
plt.figure(figsize=(8, 6))
plt.scatter(generated_tsne[:, 0], generated_tsne[:, 1], label='Generated Embeddings', alpha=0.5)
plt.scatter(ground_truth_tsne[:, 0], ground_truth_tsne[:, 1], label='Ground Truth Embeddings', alpha=0.5)
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('t-SNE of Generated vs Ground Truth Embeddings')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities = cosine_similarity(generated_embeddings, ground_truth_embeddings)
cosine_similarities_diag = np.diag(cosine_similarities)
average_cosine_similarity = np.mean(cosine_similarities_diag)

In [ ]:
average_cosine_similarity